# Data Collection Pipeline
Student Group 3 - Kickstarter Campaign Prediction

In [97]:
import pandas as pd 
import os 
import json
import re

# Data Import

## Import data from scraping from Kickstarter GraphQL endpoint

In [98]:
start = "/Users/ivankoh/Downloads/bt4222 scraped combined/"

with open(start + "0_44134_story.json", "r") as f:
    story_list1 = json.load(f)
with open(start + "0_44134_risk.json", "r") as f:
    risk_list1 = json.load(f)
with open(start + "44135_81404_story.json", "r") as f:
    story_list2 = json.load(f)
with open(start + "44135_81404_risk.json", "r") as f:
    risk_list2 = json.load(f)
with open(start + "81404_122106_story.json", "r") as f:
    story_list3 = json.load(f)
with open(start + "81404_122106_risk.json", "r") as f:
    risk_list3 = json.load(f)
with open(start + "162808_203510_story.json", "r") as f:
    story_list5 = json.load(f)
with open(start + "162808_203510_risk.json", "r") as f:
    risk_list5 = json.load(f)

story_list = story_list1 + story_list2
risk_list = risk_list1 + risk_list2

## Import data from Web Robots (Web scraping service with public datasets)

In [99]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
robot_df = pd.concat(map(pd.read_csv, filepaths))
robot_df = robot_df.drop_duplicates(subset=['id'])
robot_df['main_url'] = robot_df['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(robot_df))

203510


## Import data from WebScraper.io Scraping (Web scraping service)

In [100]:
webscraper_df = pd.read_excel("/Users/ivankoh/Downloads/bt4222 scraped combined/kickstarter_by_categories.xlsx")
graph_df1 = pd.read_csv("/Users/ivankoh/Downloads/bt4222 scraped combined/bandy.csv")
graph_df2 = pd.read_csv("/Users/ivankoh/Downloads/bt4222 scraped combined/merged_ivan_valentin.csv")
graph_df3 = pd.read_csv("/Users/ivankoh/Downloads/bt4222 scraped combined/df1_radell.csv")

graph_df2_mapping = graph_df2[['ivan_index', 'valentin_index']]
print(len(webscraper_df))
webscraper_df = webscraper_df.drop_duplicates('Link-href')
print(len(webscraper_df))

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_24477/618216799.py:4: DtypeWarning: Columns (36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  graph_df3 = pd.read_csv("/Users/ivankoh/Downloads/bt4222 scraped combined/df1_radell.csv")


50300
47341


# Data Merge

## Merge GraphQL Data (Story + Risk) with Webrobots Dataset to get Dataset X

In [101]:
# Generating Webrobots + GraphQL datasets

# graph1 + webrobots
df_robot_join = robot_df.merge(graph_df1, on="id", how="left")
df_robot_join['final_index'] = range(len(df_robot_join))

if len(story_list) == len(risk_list):
    for i, item in enumerate(story_list):
        df_robot_join.loc[df_robot_join['final_index'] == i, ['story']] = story_list[i]
        df_robot_join.loc[df_robot_join['final_index'] == i, ['risk']] = risk_list[i]
else:
    print("check story and risk list")

# graph3 + webrobots
graph_df3['graph_df3_index'] = range(len(graph_df3))
graph_df3 = graph_df3[(graph_df3['graph_df3_index'] >= 162808) & (graph_df3['graph_df3_index'] < 203510)]
graph_df3 = graph_df3[['graph_df3_index', 'id']]
graph_df3_range = df_robot_join[(df_robot_join['final_index'] >= 162808) & (df_robot_join['final_index'] < 203510)]
graph_df3_merged = graph_df3_range.merge(graph_df3,on='id',how='inner')

graph_df3_mapping = graph_df3_merged[['final_index', 'graph_df3_index']].sort_values('graph_df3_index')
base_num = 162808

for idx, row in graph_df3_mapping.iterrows():
    df_robot_join.loc[df_robot_join['final_index'] == row['final_index'], ['story']] = story_list5[row['graph_df3_index'] - base_num]
    df_robot_join.loc[df_robot_join['final_index'] == row['final_index'], ['risk']] = risk_list5[row['graph_df3_index'] - base_num]

# graph2 + webrobots    
graph_df2_mapping = graph_df2_mapping.sort_values('valentin_index')
base_num = 81404

for idx, row in graph_df2_mapping.iterrows():
    df_robot_join.loc[df_robot_join['final_index'] == row['ivan_index'], ['story']] = story_list3[row['valentin_index'] - base_num]
    df_robot_join.loc[df_robot_join['final_index'] == row['ivan_index'], ['risk']] = risk_list3[row['valentin_index'] - base_num]

# Get dataset_x: Exclude graphql data that could not fully merge with webrobots
dataset_x = df_robot_join[pd.notnull(df_robot_join['story'])]

## Merge Final GraphQL Dataset (Story + Risk) with Webscraper Dataset (Rewards)

In the proposal, the Final GraphQL Dataset is known as Dataset X, and the Webscraper Dataset is known as Dataset Y 

In [102]:
# parsing join key 
dataset_y = webscraper_df
dataset_y['Link-href'] = dataset_y['Link-href'].apply(lambda x: str(x)[:str(x).find("?ref")])
dataset_x['main_url'] = dataset_x['main_url'].apply(lambda x: str(x)[:str(x).find("?ref")])
combined_dataset = dataset_y.merge(dataset_x, left_on="Link-href", right_on="main_url", how="left")

## FILL IN MISSING DATA HERE
combined_dataset.to_csv("kickstart_data_merged_with_empty.csv")

# Exclude data in dataset X that could not fully merge with dataset Y
final_combined_dataset = combined_dataset[pd.notnull(combined_dataset['story'])] 

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_24477/4201109396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_x['main_url'] = dataset_x['main_url'].apply(lambda x: str(x)[:str(x).find("?ref")])


In [103]:
print(len(combined_dataset))
print(len(final_combined_dataset))

47341
31489


In [137]:
combined_dataset[combined_dataset['Title'] == '"Le Storie Di Selot" Colonna Sonora ispirata al romanzo'][['story','risk']]

,story,risk
30585,,


In [117]:
combined_dataset[pd.isnull(combined_dataset['story'])]

,Unnamed: 0_x,web-scraper-order,web-scraper-start-url,Title,Description,Status,Pledged-Amount,Funded-percent,Time left,Sphere,...,usd_exchange_rate,usd_pledged,usd_type,main_url,Unnamed: 0_y,ivan_index,blurb_y,story,risk,final_index
0,0,1664695459-4345,https://www.kickstarter.com/discover/advanced?...,Poly Polly: Customised multilingual children's...,"Board books for young children, customised wit...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97,1664695465-4442,https://www.kickstarter.com/discover/advanced?...,Furqan's First Flat Top,"A bi-lingual story about a young boy, his firs...",successful,NaN,116.085714,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,98,1664695470-4443,https://www.kickstarter.com/discover/advanced?...,"Oh Oh, Baby Boy! The Makings of Engaged Father...",Think of Fatherhood's potential... & let's mak...,successful,NaN,116.924593,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99,1664695476-4444,https://www.kickstarter.com/discover/advanced?...,Becoming Global: Bilingual Ukrainian-English P...,The idea of this project is to publish 2 bilin...,successful,NaN,100.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,101,1664695488-4446,https://www.kickstarter.com/discover/advanced?...,Darwin’s On the Origin of Species: A Picture B...,A picture book adaptation of the classical On ...,successful,NaN,2585.105263,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47336,127,1664440416-2517,https://www.kickstarter.com/discover/advanced?...,LOSING IT - THE MUSICAL,"A kitchen sink musical about memory, music and...",live,"£1,856",14.848000,21days to go,Musical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47337,128,1664440422-2518,https://www.kickstarter.com/discover/advanced?...,Opera Rock Gran Estrella,"Obra de Teatro Musical, basada en la tradición...",live,MX$ 18,0.001184,33hours to go,Musical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47338,129,1664440429-2519,https://www.kickstarter.com/discover/advanced?...,Had It Blue a new musical,A musical fable told to help inspire those dea...,live,"$4,476",127.885714,11days to go,Musical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47339,130,1664440434-2520,https://www.kickstarter.com/discover/advanced?...,The Catalogue of Sexual Anxieties,"Creation of a fun , vibrant, and useful intern...",live,"€1,092",14.560000,31days to go,Musical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Export

In [104]:
final_combined_dataset.to_csv("kickstarter_data_merged.csv")

In [138]:
print(sorted(final_combined_dataset.columns))
print(len(final_combined_dataset))

['Description', 'Funded-percent', 'Image-src', 'Link', 'Link-href', 'Location', 'Pledged-Amount', 'Sphere', 'Status', 'Time left', 'Title', 'Unnamed: 0_x', 'Unnamed: 0_y', 'backer-amount', 'backers_count', 'blurb_x', 'blurb_y', 'category', 'commentCount', 'converted_pledged_amount', 'country', 'country_displayable_name', 'created_at', 'creator', 'currency', 'currency_symbol', 'currency_trailing_code', 'current_currency', 'deadline', 'description_risks', 'description_story', 'disable_communication', 'final_index', 'friends', 'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred', 'ivan_index', 'launched_at', 'location', 'main_url', 'name', 'permissions', 'photo', 'pledged', 'profile', 'rewards', 'risk', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at', 'static_usd_rate', 'story', 'updateCount', 'urls', 'usd_exchange_rate', 'usd_pledged', 'usd_type', 'video', 'web-scraper-order', 'web-scraper-start-url']
31489


In [106]:
sub = final_combined_dataset[['Title', 'Description', 'Link-href', 'video', 'rewards', 'blurb_x', 'category', 'country',
      'created_at', 'deadline', 'goal', 'id', 'launched_at', 'location', 'name', 'staff_pick', 'state', 'story', 'risk']]

In [107]:
sub[pd.isnull(final['Description'])]
sub[pd.isnull(final['country'])]
sub[pd.isnull(final['state'])]
sub[pd.isnull(final['deadline'])]
sub[pd.isnull(final['created_at'])]
sub[pd.isnull(final['launched_at'])]
sub[pd.isnull(final['blurb_x'])]
sub[pd.isnull(final['staff_pick'])]
sub[pd.isnull(final['rewards'])]
sub[pd.isnull(final['story'])]


,Title,Description,Link-href,video,rewards,blurb_x,category,country,created_at,deadline,goal,id,launched_at,location,name,staff_pick,state,story,risk


In [139]:
sub[pd.isnull(final_combined_dataset['location'])]

,Title,Description,Link-href,video,rewards,blurb_x,category,country,created_at,deadline,goal,id,launched_at,location,name,staff_pick,state,story,risk
708,The Pixel Project: Impel an Aspiring Photojour...,Aspiring photojournalist seeks DSLR. Must love...,https://www.kickstarter.com/projects/226599306...,https://v2.kickstarter.com/1664678017-8qlpRFNG...,"[{""rewards"":""Pledge US$ 10 or more\n\nAbout S$...",Aspiring photojournalist seeks DSLR. Must love...,"{""id"":358,""name"":""Photo"",""analytics_name"":""Pho...",US,1.259796e+09,1.267572e+09,2000.0,1.885167e+09,1.259802e+09,NaN,The Pixel Project: Impel an Aspiring Photojour...,False,successful,"To see more work, please visit my website: ww....",
2049,On the Road Again: Asobi Seksu Goes on Tour,"We're Asobi Seksu, a band from New York City, ...",https://www.kickstarter.com/projects/549811469...,https://v2.kickstarter.com/1664121165-ebOqRzXO...,"[{""rewards"":""Pledge US$ 1 or more\n\nAbout S$ ...","We're Asobi Seksu, a band from New York City, ...","{""id"":42,""name"":""Pop"",""analytics_name"":""Pop"",""...",US,1.256781e+09,1.262322e+09,7500.0,1.761553e+09,1.257286e+09,NaN,On the Road Again: Asobi Seksu Goes on Tour,True,successful,There are some very exciting things going on f...,
3245,Offline Wikipedia iPhone app,This is a project to create a free iPhone app ...,https://www.kickstarter.com/projects/dphiffer/...,NaN,"[{""rewards"":""Pledge US$ 1 or more\n\nAbout US$...",This is a project to create a free iPhone app ...,"{""id"":51,""name"":""Software"",""analytics_name"":""S...",US,1.240638e+09,1.247555e+09,99.0,7.272860e+05,1.240674e+09,NaN,Offline Wikipedia iPhone app,True,successful,This is a project to create a free iPhone app ...,NaN
3258,MakerBeam: An Open Source Building Kit,Is it the Erector Set of the Internet Age? Or ...,https://www.kickstarter.com/projects/mnemnion/...,https://v2.kickstarter.com/1664396017-htnGnisi...,"[{""rewards"":""Pledge US$ 1 or more\n\nAbout US$...",Is it the Erector Set of the Internet Age? Or ...,"{""id"":51,""name"":""Software"",""analytics_name"":""S...",US,1.251584e+09,1.256365e+09,10000.0,2.925740e+08,1.252162e+09,NaN,MakerBeam: An Open Source Building Kit,True,successful,MakerBeam is a toy and tool for the open sourc...,
3269,"MORE JACQUES COUSTEAU, LESS HUCK FINN",Help me get the gear needed to keep me safe ro...,https://www.kickstarter.com/projects/eerichmon...,https://v2.kickstarter.com/1664456797-LNNL567D...,"[{""rewards"":""Pledge US$ 5 or more\n\nAbout US$...",Help me get the gear needed to keep me safe ro...,"{""id"":51,""name"":""Software"",""analytics_name"":""S...",US,1.266284e+09,1.271660e+09,5000.0,8.415850e+08,1.268067e+09,NaN,"MORE JACQUES COUSTEAU, LESS HUCK FINN",True,successful,OVERVIEW\nA little over 8 months ago I launche...,NaN
3273,DIA - Designing Interface Architecture (Phase ...,A series of workshops for architecture student...,https://www.kickstarter.com/projects/fabberz/d...,https://v2.kickstarter.com/1664431348-IcKsuH3g...,"[{""rewards"":""Pledge US$ 1 or more\n\nAbout US$...",A series of workshops for architecture student...,"{""id"":51,""name"":""Software"",""analytics_name"":""S...",US,1.260277e+09,1.263085e+09,7000.0,5.912374e+08,1.260383e+09,NaN,DIA - Designing Interface Architecture (Phase ...,True,successful,FABberz (www.fabberz.com) and LaN (www.livearc...,NaN
5351,The Work Office (TWO),The Work Office (TWO) is a multidisciplinary a...,https://www.kickstarter.com/projects/206989684...,NaN,"[{""rewards"":""Pledge US$ 57 or more\n\nAbout S$...",The Work Office (TWO) is a multidisciplinary a...,"{""id"":20,""name"":""Conceptual Art"",""analytics_na...",US,1.252964e+09,1.262322e+09,1175.0,1.426771e+09,1.255060e+09,NaN,The Work Office (TWO),False,successful,"For more information, visit www.theworkoffice....",
5551,The Tumbleweed Collective: Revealing America O...,Each of the fifty states is represented by one...,https://www.kickstarter.com/projects/18070994/...,https://v2.kickstarter.com/1663789583-hQSSKc